# Big Data Project - Simulated Transactions - 21/22

<strong>Andrea Rettaroli</strong> - 0000977930


### Simulated Transactions - Dataset Analysis

The goal of this notebook is to analyze and extract some useful informations from [kaggle simulated-transactions dataset](https://www.kaggle.com/datasets/conorsully1/simulated-transactions). 

The dataset contains ~22GB of data that represents random transactions.
[Notebook used to generate data](https://github.com/conorosully/medium-articles/blob/master/src/transaction_data_generator.ipynb).

Transactions are generated for 75,000 customers and are classified into 12 expenditure types:

- Groceries
- Clothing
- Housing
- Education
- Health
- Motor/Travel
- Entertainment
- Gambling
- Savings
- Bills and Utilities
- Tax
- Fines

Each transaction is represented by 10 features/columns:

- <strong> CUST_ID</strong>: unique ID for every customer
- <strong> START_DATE</strong>: the month the customer started making transactions
- <strong> END_DATE</strong>: the month the customer stopped making transactions
- <strong> TRANS_ID</strong>: unique ID for every transaction
- <strong> DATE</strong>: the date of the transaction
- <strong> YEAR</strong>: the year of the transaction
- <strong> MONTH</strong>: the month of the transaction
- <strong> DAY</strong>: the day of the transaction
- <strong> EXP_TYPE</strong>: the expenditure type (listed above)
- <strong> AMOUNT</strong>: the amount of the transaction in dollars $

# Cluster configuration

Let's start by eneble to connetct to spark ui and setting the proper cluster configuration as saw in labs. 

- 3 executors with 3 cores each (leave 1 for daemons; and there's also the AMP)
- 8G of memory per executor (recommended 11G, but it exceeds YARN's default maximum allowed in this EMR cluster)


In [1]:
sc.applicationId
"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1676214966471_0005,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: String = application_1676214966471_0005
res2: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1676214966471_0005/


In [2]:
%%configure -f
{"executorMemory":"8G", "numExecutors":3, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1676214966471_0006,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1676214966471_0006,spark,idle,Link,Link,✔


# Dataset Preprocessing 

After an analysis of the dataset I decided to:
- Removing the **DATE** column because it is the aggregation of the colums year, month and day that are more useful for the next jobs.
- creating some smaller datasets to test the correctness of the functions.


In [3]:
// Name of bucket in s3
val bucketname = "unibo-bd2122-arettaroli"
// Paths of datasets
// S3 path of simulated transactions dataset
val s3_path_dataset = "s3a://"+bucketname+"/exam-dataset/transactions.csv"
// S3 path of cleaned dataset without column "date"
val s3_path_dataset_cleaned = "s3a://"+bucketname+"/exam-dataset/transactions-cleaned.csv"
// S3 path of dataset without column "date" with 30% of data for optimization
val s3_path_dataset_small = "s3a://"+bucketname+"/exam-dataset/transactions-small.csv"
// S3 path of dataset without column "date" with 15 rows (to test the correct functioning of the jobs)
val s3_path_dataset_smallest = "s3a://"+bucketname+"/exam-dataset/transactions-smallest.csv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2122-arettaroli
s3_path_dataset: String = s3a://unibo-bd2122-arettaroli/exam-dataset/transactions.csv
s3_path_dataset_cleaned: String = s3a://unibo-bd2122-arettaroli/exam-dataset/transactions-cleaned.csv
s3_path_dataset_small: String = s3a://unibo-bd2122-arettaroli/exam-dataset/transactions-small.csv
s3_path_dataset_smallest: String = s3a://unibo-bd2122-arettaroli/exam-dataset/transactions-smallest.csv


**ALERT:** The next three cells need to be executed only the first time.

In [ ]:
// Read data from the simulated transactions dataset, drops the column DATE (_c4) then save to another csv.
spark.
  read.
  csv(s3_path_dataset).
  drop("_c4").
  write.
  csv(s3_path_dataset_cleaned)

In [ ]:
// Read data from csv, limit the number of rows considering the 30% of data for optimization
// Number of transactions: 261969719 => 30% of transactions is : 78.590.915,7 => 78500000 
spark.
  read.
  csv(s3_path_dataset_cleaned).
  limit(78500000). 
  write.
  csv(s3_path_dataset_small)

In [ ]:
// Save a small set of data for check the results of following jobs
spark.
  read.
  csv(s3_path_dataset_cleaned).
  limit(10).
  write.
  csv(s3_path_dataset_smallest)

# Dataset preparation


In order to read each transaction is useful to define an object **TransactionsParser** to parse the transactions from the csv file. 

Each value in the csv is divided by ',' (comma) so for each line it's possible to retrieve values splitting on comma.


In [4]:
import java.util.Calendar
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner

object TransactionsParser {
  // Each value in the csv is divided by ','
  val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    
  type CustomerId = String
  type StartDate = String
  type EndDate = String
  type TransactionId = String 
  // type TransactionDate = String
  type Year = Int
  type Month = Int
  type Day = Int
  type ExpenditureType = String
  type Amount = Double
  
  def parseTransactionLine(line: String): Option[(CustomerId, StartDate, EndDate, TransactionId, Year, Month, Day, ExpenditureType, Amount)] = {
    try {
      val input = line.split(commaRegex) // Splitting on comma
      if (input(0) == "CUST_ID") { // To discard the headers column line
          None
      }
      Some(input(0).trim, input(1).trim, input(2).trim, input(3).trim, 
           input(4).trim.toInt, input(5).trim.toInt, input(6).trim.toInt, input(7).trim, input(8).trim.toDouble)
      
    } catch {
      case _: Exception => None
    }
  }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.util.Calendar
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner
defined object TransactionsParser


#### Analysis and considerations on enforcement

Using SparkUI is possible to see that the dataset size without date column  is  around 18 GB. 
The strategy decided to save the data is: serialize them in memory and disk, in this way when the memory is full, the disk is used.

By default Spark creates 665 partitions for this dataset which is a lot because each task should take at least 100-200ms. This mean that it is better to partition less with the same total execution time.

In [5]:
import org.apache.spark.storage.StorageLevel._

// Each row is flatMapped with parseTransactionLine method

//val rddTransactionsOriginal = sc.textFile(s3_path_dataset).flatMap(TransactionsParser.parseTransactionLine)
//val rddTransactions = sc.textFile(s3_path_dataset_small).flatMap(TransactionsParser.parseTransactionLine).coalesce(90)
val rddTransactions = sc.textFile(s3_path_dataset_cleaned).flatMap(TransactionsParser.parseTransactionLine).coalesce(300)
// Persist on memory and disk with serialization
val diskMemoryRdd = rddTransactions.persist(MEMORY_AND_DISK_SER) 

// Same for the smallest (10 rows)
val rddTransactions_smallest = sc.textFile(s3_path_dataset_smallest).flatMap(TransactionsParser.parseTransactionLine)
// Persist on memory and disk with serialization
val diskMemoryRdd_smallest = rddTransactions_smallest.persist(MEMORY_AND_DISK_SER) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.storage.StorageLevel._
rddTransactions: org.apache.spark.rdd.RDD[(TransactionsParser.CustomerId, TransactionsParser.StartDate, TransactionsParser.EndDate, TransactionsParser.TransactionId, TransactionsParser.Year, TransactionsParser.Month, TransactionsParser.Day, TransactionsParser.ExpenditureType, TransactionsParser.Amount)] = CoalescedRDD[3] at coalesce at <console>:40
diskMemoryRdd: rddTransactions.type = CoalescedRDD[3] at coalesce at <console>:40
rddTransactions_smallest: org.apache.spark.rdd.RDD[(TransactionsParser.CustomerId, TransactionsParser.StartDate, TransactionsParser.EndDate, TransactionsParser.TransactionId, TransactionsParser.Year, TransactionsParser.Month, TransactionsParser.Day, TransactionsParser.ExpenditureType, TransactionsParser.Amount)] = MapPartitionsRDD[6] at flatMap at <console>:37
diskMemoryRdd_smallest: rddTransactions_smallest.type = MapPartitionsRDD[6] at flatMap at <console>:37


In [8]:
//data preview
diskMemoryRdd_smallest.
    collect().
    foreach(transaction => print(transaction + "\n"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(CPVZ2MIAO3,2010-07-01,2017-01-01,T18P9KZ3O350W73,2012,4,5,Entertainment,21.48)
(CPVZ2MIAO3,2010-07-01,2017-01-01,T2N065QH3L11AZS,2015,5,24,Entertainment,17.25)
(CPVZ2MIAO3,2010-07-01,2017-01-01,TRNQXHP2B5T069M,2014,3,5,Education,412.9)
(CPVZ2MIAO3,2010-07-01,2017-01-01,T9PQMGLRKOAI2SJ,2012,9,13,Entertainment,19.18)
(CPVZ2MIAO3,2010-07-01,2017-01-01,TS5UMAAB2WWPNXW,2016,5,13,Entertainment,18.66)
(CPVZ2MIAO3,2010-07-01,2017-01-01,T48ZCBVO5UW71NC,2013,12,9,Groceries,63.61)
(CPVZ2MIAO3,2010-07-01,2017-01-01,T8K9FUYYV2OY9WB,2015,12,4,Entertainment,45.66)
(CPVZ2MIAO3,2010-07-01,2017-01-01,T2Y5VJ9D579QGTB,2012,8,5,Entertainment,49.35)
(CPVZ2MIAO3,2010-07-01,2017-01-01,T13QP1402CVF9PW,2012,9,22,Groceries,56.19)
(CPVZ2MIAO3,2010-07-01,2017-01-01,TY7N8BTA0ASGXO9,2011,7,18,Groceries,64.94)


**DEBUG: use to see RDDs partitions status**: the cell below show the RDDs partiotions status.

In [ ]:
sc.getRDDStorageInfo.foreach(x=> print(x + "\n")) 

**Use only to clean RDDs in memory**: the cell below clean the RDDs in memory.

In [ ]:
sc.getPersistentRDDs.foreach(_._2.unpersist())

# Dataset exploration

In this section various exploration queries are performed on the dataset but also more complex jobs to find useful information from data.

To improve drastically the performances data used in different jobs will be cached in memory and disk.

In [8]:
// Caching of customers, amount, year, expenditure type and amount for following jobs to improve performance
val cachedCustomer = rddTransactions.map(x=>x._1).persist(MEMORY_AND_DISK_SER)
val cachedAmount = rddTransactions.map(x=>x._9).persist(MEMORY_AND_DISK_SER)
val cachedYear = rddTransactions.map(x=>x._5).persist(MEMORY_AND_DISK_SER)
val cachedExpenditureType = rddTransactions.map(x=>x._8).persist(MEMORY_AND_DISK_SER)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cachedCustomer: org.apache.spark.rdd.RDD[TransactionsParser.CustomerId] = MapPartitionsRDD[7] at map at <console>:32
cachedAmount: org.apache.spark.rdd.RDD[TransactionsParser.Amount] = MapPartitionsRDD[8] at map at <console>:31
cachedYear: org.apache.spark.rdd.RDD[TransactionsParser.Year] = MapPartitionsRDD[9] at map at <console>:31
cachedExpenditureType: org.apache.spark.rdd.RDD[TransactionsParser.ExpenditureType] = MapPartitionsRDD[10] at map at <console>:31


### Exploring dataset dimentions

To know the size of the dataset entities:
1. How many transactions? 
2. How many distinct customers?
3. How many distinct expenditure type?
4. How many distinct years?
5. Min amount and max ammount?
6. From what year to what year?

In [13]:
"1. Number of transactions: " + diskMemoryRdd.count() // Each row is a transaction => 261.969.719 already without header line
"2. Number of distinct customers: " + cachedCustomer.distinct().count() // 75000
"3. Number of distinct expenditure type: " + cachedExpenditureType.distinct().count() // 12 
"4. Number of distinct years: " + cachedYear.distinct().count() // 11
"5. Range of amount: " + cachedAmount.min() + " $ to " + cachedAmount.max() +" $" //0.12 to 6519.61
"6. From year: " + cachedYear.min() + " to " + cachedYear.max() // 2010 to 2020

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res27: String = 1. Number of transactions: 261969719
res28: String = 2. Number of distinct customers: 75000
res29: String = 3. Number of distinct expenditure type: 12
res30: String = 4. Number of distinct years: 11
res31: String = 5. Range of amount: 0.12 $ to 6519.61 $
res32: String = 6. From year: 2010 to 2020


Learn more about this dataset by answering more complex questions such as:

7. which are the years listed in the dataset in ascending order?
8. What is the average amount?
9. What is the average amount calculate for every year?
10. What is the average amount for each type of expenditure?
11. What is the maximum amount for each type of expenditure?

In [14]:
//7. which are the years listed in the dataset in ascending order?
cachedYear.
    distinct().
    collect().
    sorted.
    foreach(year => print(year + "\n"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [15]:
//8. What is the average amount?
"Amount on average: " + cachedAmount.mean()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res36: String = Amount on average: 81.56297388367233


In [16]:
//9. What is the average amount calculate for every year?
diskMemoryRdd.
    map(x => (x._5, x._9)). //take YEAR and AMOUNT
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)). //sum AMOUNT on fist value, and count summed AMOUNTs on the second, end it aggregate partitions
    map({case(k,v)=>(k,v._1/v._2)}). //calculate the average based on YEAR(key)
    sortByKey(). //order on YEAR
    collect().
    foreach(result => print(result + "\n"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(2010,58.13171661388978)
(2011,59.74314431061088)
(2012,62.61107660724443)
(2013,65.92462474542249)
(2014,69.75024925871848)
(2015,73.98090515674579)
(2016,78.64610882393872)
(2017,83.63442584685858)
(2018,89.35581974762611)
(2019,96.05548560402433)
(2020,103.33282605610562)


In [17]:
// Caching the pair EXPENDITURE_TYPE - AMOUNT for following jobs to improve performance 
val cachedExpenditureTypeAmount = rddTransactions.map(x => (x._8, x._9)).persist(MEMORY_AND_DISK_SER)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cachedExpenditureTypeAmount: org.apache.spark.rdd.RDD[(TransactionsParser.ExpenditureType, TransactionsParser.Amount)] = MapPartitionsRDD[39] at map at <console>:32


In [18]:
//10. What is the average amount for each type of expenditure?
cachedExpenditureTypeAmount.
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)). //sum AMOUNT on fist value, and count summed AMOUNTs on the second, end it aggregate partitions
    map({case(k,v)=>(k,v._1/v._2)}). //calculate the average based on EXPENDITURE_TYPE(key)
    sortBy(_._2, false). //order by AMOUNT descending 
    collect().
    foreach(result => print(result + "\n"))

// In this way is also possible to view which expenditure type is more expensive on average

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(Housing,1558.7478390875353)
(Tax,412.95141059769145)
(Education,281.1921104205164)
(Savings,223.37088318474528)
(Bills and Utilities,208.28440022266733)
(Clothing,179.36878004462213)
(Health,159.31446095332151)
(Fines,159.2007908590806)
(Motor/Travel,133.38064934051437)
(Gambling,105.02011185360804)
(Groceries,80.30824109361194)
(Entertainment,24.031486539717367)


In [20]:
//11. What is the maximum amount for each type of expenditure?
cachedExpenditureTypeAmount.
    reduceByKey((x,y)=>{if(x<y) y else x}). //take the maximum AMMOUNT for each EXPENDITURE_TYPE
    sortBy(_._2, false). //reorder by AMOUNT descending 
    collect().
    foreach(result => print(result + "\n"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(Housing,6519.61)
(Motor/Travel,6334.35)
(Clothing,4319.49)
(Education,2787.69)
(Savings,1996.85)
(Groceries,1952.39)
(Health,1816.22)
(Bills and Utilities,1489.54)
(Tax,1333.29)
(Entertainment,1002.84)
(Fines,988.5)
(Gambling,870.31)


### Analyze transactions

Analyze transactions more in deep answering the follow questions:

1. What is the average of transactions for customer?
2. What is the average of transactions for year?
3. What is the average of transactions for expenditure type?
4. Which is the year with the most number of transactions?
5. Which is the month with the most number of transactions?
6. Which is the year-month with the most number of transactions?


In [21]:
// For following jobs to improve performance 
// Caching the pair CUSTOMER - NUMBER OF TRANSACTIONS 
val cachedCustomerTransactions = rddTransactions.map(x => (x._1, 1)).reduceByKey(_+_).persist(MEMORY_AND_DISK_SER) //create pair (customer_id, 1) and sum reducing by customer_id
// Caching the pair YEAR - NUMBER OF TRANSACTIONS
val cachedYearTransactions = rddTransactions.map(x => (x._5, 1)).reduceByKey(_+_).persist(MEMORY_AND_DISK_SER) //create pair (year, 1) and  sum reducing by year
// Caching the pair EXPENDITURE TYPE - NUMBER OF TRANSACTIONS
val cachedExpenditureTypeTransactions = rddTransactions.map(x => (x._8, 1)).reduceByKey(_+_).persist(MEMORY_AND_DISK_SER) //create pair (expenditure_type, 1) and sum reducing by expenditure_type

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cachedCustomerTransactions: org.apache.spark.rdd.RDD[(TransactionsParser.CustomerId, Int)] = ShuffledRDD[60] at reduceByKey at <console>:33
cachedYearTransactions: org.apache.spark.rdd.RDD[(TransactionsParser.Year, Int)] = ShuffledRDD[62] at reduceByKey at <console>:32
cachedExpenditureTypeTransactions: org.apache.spark.rdd.RDD[(TransactionsParser.ExpenditureType, Int)] = ShuffledRDD[64] at reduceByKey at <console>:32


In [22]:
//1. What is the average of transactions for customer?

// Two methods
// 1st method => easier
"Number of transactions per customer: " + diskMemoryRdd.count() / cachedCustomer.distinct().count()

// 2nd method
val avgTransactionsPerCustomer = cachedCustomerTransactions.
    aggregate((0,0))((a,v)=>(a._1+v._2, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)) //crate pair (Customer_transactions, counter_of_customers) and sum each pair
   

"Number of transactions per customer: " + (avgTransactionsPerCustomer._1/avgTransactionsPerCustomer._2) // (total transactions / number of different customers)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res57: String = Number of transactions per customer: 3492
avgTransactionsPerCustomer: (Int, Int) = (261969719,75000)
res62: String = Number of transactions per customer: 3492


In [23]:
//2. What is the average of transactions for year?

// Two methods
// 1st method => easier
"Number of transactions per year: " + diskMemoryRdd.count() / cachedYear.distinct().count()

// 2nd method
val avgTransactionsPerYear = cachedYearTransactions.
    aggregate((0,0))((a,v)=>(a._1+v._2, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)) //crate pair (transactions_per_year, counter_of_years) and sum each pair

"Number of transactions per year: " + (avgTransactionsPerYear._1/avgTransactionsPerYear._2) // (total transactions / number of different years)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res67: String = Number of transactions per year: 23815429
avgTransactionsPerYear: (Int, Int) = (261969719,11)
res71: String = Number of transactions per year: 23815429


In [24]:
//3. What is the average of transactions for expenditure type?

// Two methods
// 1st method => simpler
"Number of transactions per expenditure type: " + diskMemoryRdd.count() / cachedExpenditureType.distinct().count()

// 2nd method
val avgTransactionsPerExpenditureType = cachedExpenditureTypeTransactions.
    aggregate((0,0))((a,v)=>(a._1+v._2, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2))

"Number of transactions per expenditure type: " + (avgTransactionsPerExpenditureType._1/avgTransactionsPerExpenditureType._2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res76: String = Number of transactions per expenditure type: 21830809
avgTransactionsPerExpenditureType: (Int, Int) = (261969719,12)
res80: String = Number of transactions per expenditure type: 21830809


In [ ]:
// Save results to a csv file
val s3_path_transaction_per_expenditure_type = "s3a://"+bucketname+"/exam-dataset/jobs/transaction-per-expenditure-type.csv"

cachedExpenditureTypeTransactions.
    coalesce(1). // put all in one partition to have only one csv file 
    toDF().
    write.
    format("csv").
    mode(SaveMode.Overwrite).
    save(s3_path_transaction_per_expenditure_type)

In [ ]:
// 4. Which is the year with the most number of transactions?

val yearWithMostTransactions = cachedYearTransactions.
    sortBy(_._2, false). //order by tatal transactions counted descending
    first()

"The year with the most number of transactions is: " + yearWithMostTransactions._1 + " with a total of : "+ yearWithMostTransactions._2 +" transactions"

In [ ]:
//5. Which is the month with the most number of transactions?
val monthWithMoreTransactions = diskMemoryRdd.map(x => (x._6,1)). //create pair (month,counter) for each transaction example: (january, 1) 
    reduceByKey(_+_).   // sum by month 
    sortBy(_._2, false). //order by tatal transactions counted descending
    first()


"The month with the most number of transactions is: " +  monthWithMoreTransactions._1 + " with a total of : "+ monthWithMoreTransactions._2

In [ ]:
//6. Which is the year-month with the most number of transactions?


val pairYearMonthWithMoreTransactions = diskMemoryRdd.map(x => ((x._5, x._6),1)).
    reduceByKey(_+_). //reduce in pair (year,month) 
    sortBy(_._2, false). //order by counter of transactions descending
    first()

"The pair Year-Month with the most number of transactions: " + pairYearMonthWithMoreTransactions

### Analyze customers

Analyze customers answering the follow questions:

1. What is the customer with the most number of transactions?
2. What is the customer with the most amount spent?


In [25]:
// 1.  What is the customer with the most number of transactions?

// use of sort + first
val customerWithMoreTransactions = cachedCustomerTransactions.
    sortBy(_._2, false). //order by Transactions descending 
    first()

"Customer: " + customerWithMoreTransactions._1 + " has made: " + customerWithMoreTransactions._2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

customerWithMoreTransactions: (TransactionsParser.CustomerId, Int) = (C0YDPQWPBJ,10369)
res85: String = Customer: C0YDPQWPBJ has made: 10369


In [6]:
// Caching the pair (Customer_id, Total_amount_spent) to improve performance for following jobs
val cachedCustomersTotalAmounts = diskMemoryRdd.map(x => (x._1, x._9)).
                                         reduceByKey(_+_). //sum AMOUNT spent for transactions of each Customer
                                         persist(MEMORY_AND_DISK_SER) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cachedCustomersTotalAmounts: org.apache.spark.rdd.RDD[(TransactionsParser.CustomerId, TransactionsParser.Amount)] = ShuffledRDD[8] at reduceByKey at <console>:33


In [10]:
//2.  What is the customer with the most amount spent?

val customerWithMostAmountSpent = cachedCustomersTotalAmounts.
    sortBy(_._2, false). //order by Total amount spent descending 
    first()

"Customer: " + customerWithMostAmountSpent._1 + " with: " + customerWithMostAmountSpent._2


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

customerWithMostAmountSpent: (TransactionsParser.CustomerId, TransactionsParser.Amount) = (CP2KXQSX9I,2310805.889999996)
res22: String = Customer: CP2KXQSX9I with: 2310805.889999996


# Classification

The goal of this project is classify customers according to their spending, assigning them a score, then add to each line the score and recalculate the total transactions on each type of expense and then graph the result with a heatmap. 
In doing this you also want to meet the following tasks: 

1. Classify each customer on the basis of how much he has spent in total using a rank from 1 to 5

2. Classify each expenditure type on the basis of how much is the total spent using a rank from 1 to 5

3. Classify each expenditure type on the basis of how much is the total spent for each customer using a rank from 1 to 5

where: 
- 1 means low 
- 2 means mid-low 
- 3 means mid
- 4 means mid-high
- 5 means high



 ### Task 1
 Classify each customer on the basis of how much he has spent in total using a rank from 1 to 5.

In [28]:
// Number of distinct customer 
val totalDistinctCustomers = cachedCustomersTotalAmounts.count() //or cachedCustomer.distinct().count()

"Total distinct customers " + totalDistinctCustomers

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

totalDistinctCustomers: Long = 75000
res93: String = Total distinct customers 75000


In [30]:
// To define the bounds of ranking 
val lowestBound = (totalDistinctCustomers * 0.2).toInt 
val lowBound = (totalDistinctCustomers * 0.4).toInt 
val mediumBound = (totalDistinctCustomers * 0.6).toInt
val highBound = (totalDistinctCustomers * 0.8).toInt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lowestBound: Int = 15000
lowBound: Int = 30000
mediumBound: Int = 45000
highBound: Int = 60000


In [31]:
val rankCustomerPerTotalAmount = cachedCustomersTotalAmounts.
    sortBy(x => x._2, true). //sort by total amount ascending
    zipWithIndex(). // gives an index 
    map{ x => //x is ((customer_id, total_amount),index)
        val rank = x._2 match { //where v is the index in the list of consumers sorted by total amount ascending = x._2(index)
            case v if (v <= lowestBound) => 1
            case v if (v <= lowBound) => 2
            case v if (v <= mediumBound) => 3
            case v if (v <= highBound) => 4
            case _ => 5
        }
        (x._1._1, x._1._2, rank) // return (customer id , taotal ammount , rank)
    }

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rankCustomerPerTotalAmount: org.apache.spark.rdd.RDD[(TransactionsParser.CustomerId, TransactionsParser.Amount, Int)] = MapPartitionsRDD[92] at map at <console>:42


In [ ]:
// Show for each customer the ranking based on the total amount 
rankCustomerPerTotalAmount.collect().foreach(result => print("customer: "+ result._1+" with: total amount spent: "+result._2+ " has rank: "+result._3 + "\n"))

In [33]:
// Save results to a csv file
val s3_path_rank_customer_per_total_amount= "s3a://"+bucketname+"/exam-dataset/jobs/rank-customer-per-total-amount.csv"

rankCustomerPerTotalAmount.
    coalesce(1).
    toDF().
    write.
    format("csv").
    mode(SaveMode.Overwrite).
    save(s3_path_rank_customer_per_total_amount)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3_path_rank_customer_per_total_amount: String = s3a://unibo-bd2122-arettaroli/exam-dataset/jobs/rank-customer-per-total-amount.csv


### Task 2
Classify each expenditure type on the basis of how much is the total spent using a rank from 1 to 5.

In [34]:
// Number of distinct expenditure type
val distinctExpenditureType = cachedExpenditureType.distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

distinctExpenditureType: Long = 12


In [35]:
// To define the bounds of ranking 
val lowestBound = (distinctExpenditureType * 0.2).toInt 
val lowBound = (distinctExpenditureType * 0.4).toInt 
val mediumBound = (distinctExpenditureType * 0.6).toInt
val highBound = (distinctExpenditureType * 0.8).toInt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lowestBound: Int = 2
lowBound: Int = 4
mediumBound: Int = 7
highBound: Int = 9


In [37]:
val rankExpenditureTypePerTotalAmount = cachedExpenditureTypeAmount. // pair EXPENDITURE_TYPE - AMOUNT
    reduceByKey(_+_). //sum of amount based on expenditure type(key)
    sortBy(x => x._2, true). //sort by total AMOUNT per expenditure type ascending
    zipWithIndex(). // gives an index 
    map{ x => //x is ((expenditure_type, total_amount),index)
        val rank = x._2 match { //where v is the index in the list of expenditure type sorted by total amount ascending = x._2(index)
            case v if (v <= lowestBound) => 1
            case v if (v <= lowBound) => 2
            case v if (v <= mediumBound) => 3
            case v if (v <= highBound) => 4
            case _ => 5
        }
        (x._1._1, x._1._2, rank) // return (expenditure type , taotal ammount , rank)
    }

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rankExpenditureTypePerTotalAmount: org.apache.spark.rdd.RDD[(TransactionsParser.ExpenditureType, TransactionsParser.Amount, Int)] = MapPartitionsRDD[116] at map at <console>:43


In [38]:
// Show for each expenditure type the ranking based on amount
rankExpenditureTypePerTotalAmount.collect().foreach(result => print(result + "\n"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(Fines,8074186.509999991,1)
(Savings,4.85263415399999E8,1)
(Gambling,4.9470059752999973E8,1)
(Education,9.591656909000006E8,2)
(Tax,1.2155885493200004E9,2)
(Health,1.236805636089999E9,3)
(Clothing,1.3922672867199986E9,3)
(Bills and Utilities,1.7967274707599993E9,3)
(Housing,2.6891112445500026E9,4)
(Entertainment,3.5645641079899993E9,4)
(Groceries,3.6450088438400044E9,5)
(Motor/Travel,3.879752319500002E9,5)


In [39]:
// Save results to a csv file
val s3_path_rank_expenditure_type_per_total_amount = "s3a://"+bucketname+"/exam-dataset/jobs/rank-expenditure-type-per-total-amount.csv"

rankExpenditureTypePerTotalAmount.
    coalesce(1).
    toDF().
    write.
    format("csv").
    mode(SaveMode.Overwrite).
    save(s3_path_rank_expenditure_type_per_total_amount)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3_path_rank_expenditure_type_per_total_amount: String = s3a://unibo-bd2122-arettaroli/exam-dataset/jobs/rank-expenditure-type-per-total-amount.csv


### Task 3
Classify each expenditure type on the basis of how much is the total spent for each customer using a rank from 1 to 5.

In [7]:
// Number of distinct customer 
val totalDistinctCustomers = cachedCustomersTotalAmounts.count() //or cachedCustomer.distinct().count()

"Total distinct customers " + totalDistinctCustomers

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

totalDistinctCustomers: Long = 75000
res20: String = Total distinct customers 75000


In [8]:
// To define the bounds of ranking 
val lowestBound = (totalDistinctCustomers * 0.2).toInt 
val lowBound = (totalDistinctCustomers * 0.4).toInt 
val mediumBound = (totalDistinctCustomers * 0.6).toInt
val highBound = (totalDistinctCustomers * 0.8).toInt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lowestBound: Int = 15000
lowBound: Int = 30000
mediumBound: Int = 45000
highBound: Int = 60000


In [9]:
// Key by customers for join, the value is the expenditure type and the amount of transaction
val rddOriginal = diskMemoryRdd.map(x => (x._1, (x._8, x._9))) //create a pair (customers_id, (expenditure_type,amount))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddOriginal: org.apache.spark.rdd.RDD[(TransactionsParser.CustomerId, (TransactionsParser.ExpenditureType, TransactionsParser.Amount))] = MapPartitionsRDD[9] at map at <console>:32


In [11]:
// Join on rddOriginal and then key by Expenditure type and Ranking and sum amounts
val ExpenditureTypePerCustomerRank = cachedCustomersTotalAmounts. // pair (customers_id,total amount spent)
    sortBy(x => x._2, true). //sort by total amount spent ascending
    zipWithIndex(). // gives an index
    map{ x => //x is ((customers_id,total amount spent),index)
        val rank = x._2 match { //where v is the index in the list of consumers sorted by total amount ascending = x._2(index)
            case v if (v <= lowestBound) => 1
            case v if (v <= lowBound) => 2
            case v if (v <= mediumBound) => 3
            case v if (v <= highBound) => 4
            case _ => 5
        }
        (x._1._1, rank) // return pair (customers_id, rank)
    }. 
    join(rddOriginal). //(customers_id, rank)  (customers_id, (expenditure_type,amount)) => (customer_id, (rank, (expenditure_type,amount))) es: (CIRTTURCZT,(4,(Entertainment,49.69)))  to see the structure you can use .take(x).foreach(result=>print(result))
    map(x => ((x._2._2._1, x._2._1), x._2._2._2)). // ((espenditure type, rank), ammount)
    reduceByKey(_+_) //=> sum amount by (espenditure type, rank) as key 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rankPerExpenditureType: org.apache.spark.rdd.RDD[((TransactionsParser.ExpenditureType, Int), TransactionsParser.Amount)] = ShuffledRDD[31] at reduceByKey at <console>:57


In [13]:
// Save results to a csv file
val s3_path_customer_rank_per_expenditure_type = "s3a://"+bucketname+"/exam-dataset/jobs/expenditure-type-per-customer-rank-per-total-amount.csv"

rankPerExpenditureType.
    map(x => (x._1._1, x._1._2, x._2)). //map in (expenditure type, ranking, amount)
    coalesce(1).
    toDF().
    write.
    format("csv").
    mode(SaveMode.Overwrite).
    save(s3_path_customer_rank_per_expenditure_type)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3_path_customer_rank_per_expenditure_type: String = s3a://unibo-bd2122-arettaroli/exam-dataset/jobs/expenditure-type-per-customer-rank-per-total-amount.csv
